In [4]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
import spacy
import time
import dataiku
import pickle
import os

In [107]:
import requests
from bs4 import BeautifulSoup

base_url = 'https://bina.az/items/{item_number}'
links = set()
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                  '(KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

url = 'https://bina.az/baki/kiraye/menziller'
page = requests.get(url, headers=headers)
page.raise_for_status()
html = BeautifulSoup(page.content, 'html.parser')

while True:
    for link in html.find_all('a', href=True):
        href = link['href']
        if '/items/' in href and '=' not in href:
            links.add(base_url.format(item_number=href.split('/')[-1]))

    next_button = html.select_one('.next a')
    if not next_button:
        break

    next_url = 'https://bina.az' + next_button['href']
    page = requests.get(next_url, headers=headers)
    page.raise_for_status()
    html = BeautifulSoup(page.content, 'html.parser')

links = list(links)
#print(len(links))  # Number of gathered links


In [108]:
len(set(links))


15065

In [10]:
import pickle
import dataiku

with open(os.path.join(dataiku.Folder('aN1a8BD3').get_path(), 'collected_link.pkl'), 'rb') as file:
    links = pickle.load(file)

In [11]:
links

['https://bina.az/items/3971067',
 'https://bina.az/items/4005977',
 'https://bina.az/items/3970414',
 'https://bina.az/items/4011976',
 'https://bina.az/items/4000983',
 'https://bina.az/items/3959544',
 'https://bina.az/items/3987526',
 'https://bina.az/items/3980543',
 'https://bina.az/items/3996900',
 'https://bina.az/items/3988553',
 'https://bina.az/items/3976519',
 'https://bina.az/items/3980044',
 'https://bina.az/items/4008278',
 'https://bina.az/items/3959464',
 'https://bina.az/items/3955229',
 'https://bina.az/items/3992780',
 'https://bina.az/items/3991360',
 'https://bina.az/items/3963695',
 'https://bina.az/items/4010083',
 'https://bina.az/items/3796310',
 'https://bina.az/items/4008990',
 'https://bina.az/items/3988012',
 'https://bina.az/items/3836089',
 'https://bina.az/items/3814142',
 'https://bina.az/items/3968993',
 'https://bina.az/items/3985217',
 'https://bina.az/items/3971087',
 'https://bina.az/items/4007777',
 'https://bina.az/items/3919016',
 'https://bina

In [12]:
# NLP processing for descriptions using spaCyb
#nlp = spacy.load('en_core_web_sm')
    
# Create a DataFrame to store scraped data
kiraye_evler = pd.DataFrame(columns=[
    'Category', 'Floor', 'Area(m²)', 'Room_count', 'Repairs', 'Price', 'Currency', 'Daily_monthly', 'Location', 'Link'
])

# Iterate through each link and scrape data
try:
    for url in links:
        try:
            # Extracting description using the function
#             description = extract_description(url)

            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                              '(KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
            }
            page = requests.get(url, headers=headers)
            page.raise_for_status()
            html = BeautifulSoup(page.content, 'html.parser')
            kiraye = html.find_all('div', class_='product-properties__column')
            price = html.find_all('div', class_='product-price')
            location=html.find_all('ul', class_='product-extras bz-d-flex bz-align-center bz-gap-15 bz-wrap-wrap')
            desc=html.find_all('div', class_='product-description__content')

            ozellikler = re.sub('<[^>]+>', ' ', str(kiraye)).split('  ')
            price = re.sub('<[^>]+>', ' ', str(price)).split('  ')
            loc=re.sub('<[^>]+>', ' ', str(location)).split('   ')
            description=re.sub('<[^>]+>', ' ', str(desc)).replace('\n', '').replace(']', '').replace('[', '').strip()

            if len(ozellikler) > 14 and len(price) > 3:
                kateqoriya = ozellikler[2]
                mertebe = ozellikler[5]
                sahe = ozellikler[8].replace('m²', '')
                otaq_sayi = ozellikler[11]
                temir = ozellikler[14]
                qiymet = price[1].replace(' ', '')
                currency = price[2]
                gunluk_ayliq = price[3].replace('/', '')
                location=str(loc[1:-1])

                # Append the data to the DataFrame
                kiraye_evler = kiraye_evler.append({
                    'Category': kateqoriya,
                    'Floor': mertebe,
                    'Area(m²)': sahe,
                    'Room_count': otaq_sayi,
                    'Repairs': temir,
                    'Price': qiymet,
                    'Currency': currency,
                    'Daily_monthly': gunluk_ayliq,
                    'Location':location,
                    'Link':url,
                    'Description': description  # Add the description column
                }, ignore_index=True)
            else:
                print(f"Some elements not found or structure changed for URL: {url}")
        except requests.RequestException as e:
            print(f"Error fetching data from {url}: {e}")
except:
    # Process descriptions using spaCy
    #kiraye_evler['Processed_Description'] = kiraye_evler['Description'].apply(lambda x: [token.text for token in nlp(x)] if x else None)
    kirayeler = dataiku.Dataset("KIRAYELER")
    kirayeler.write_with_schema(kiraye_evler)
    

#kiraye_evler['Processed_Description'] = kiraye_evler['Description'].apply(lambda x: [token.text for token in nlp(x)] if x else None)
kirayeler = dataiku.Dataset("KIRAYELER")
kirayeler.write_with_schema(kiraye_evler)

# Print or use the 'kiraye_evler' DataFrame with the added processed description column
#print(kiraye_evler)


Error fetching data from https://bina.az/items/3971067: 403 Client Error: Forbidden for url: https://bina.az/items/3971067
Error fetching data from https://bina.az/items/4005977: 403 Client Error: Forbidden for url: https://bina.az/items/4005977
Error fetching data from https://bina.az/items/3970414: 403 Client Error: Forbidden for url: https://bina.az/items/3970414
Error fetching data from https://bina.az/items/4011976: 403 Client Error: Forbidden for url: https://bina.az/items/4011976
Error fetching data from https://bina.az/items/4000983: 403 Client Error: Forbidden for url: https://bina.az/items/4000983
Error fetching data from https://bina.az/items/3959544: 403 Client Error: Forbidden for url: https://bina.az/items/3959544
Error fetching data from https://bina.az/items/3987526: 403 Client Error: Forbidden for url: https://bina.az/items/3987526
Error fetching data from https://bina.az/items/3980543: 403 Client Error: Forbidden for url: https://bina.az/items/3980543
Error fetching d

In [6]:
kiraye_evler

,Category,Floor,Area(m²),Room_count,Repairs,Price,Currency,Daily_monthly,Location,Link,Description,Processed_Description
0,Yeni tikili,14 / 18,200,4,var,900,AZN,ay,"['Şərq bazarı', ' Nərimanov r.']",https://bina.az/items/3761469,Iki ayin pulu onceden istenilir. Новостройка! ...,"[Iki, ayin, pulu, onceden, istenilir, ., Новос..."
1,Yeni tikili,16 / 22,100,3,var,699,AZN,ay,['Yasamal r.'],https://bina.az/items/3997875,20 Yanvarda kompleksdə 22 mərtəbəli binanın 16...,"[20, Yanvarda, kompleksdə, 22, mərtəbəli, bina..."
2,Köhnə tikili,4 / 5,67,3,var,550,AZN,ay,"['Memar Əcəmi m.', ' Nəsimi r.', ' 2-ci mikror...",https://bina.az/items/3982938,Memar Əcəmi metro stansiyasının yanında 5 mərt...,"[Memar, Əcəmi, metro, stansiyasının, yanında, ..."
3,Yeni tikili,12 / 18,90,2,var,750,AZN,ay,"['Təhsil Nazirliyi', ' Şah İsmayıl Xətai m.',...",https://bina.az/items/3973771,"Nərimanov rayonu Təhsil Nazirliyinin yanında, ...","[Nərimanov, rayonu, Təhsil, Nazirliyinin, yanı..."
4,Yeni tikili,12 / 15,125,3,var,750,AZN,ay,"['8 Noyabr m.', ' Nəsimi r.']",https://bina.az/items/3998662,"Tibb Universitetindən 10 dəqiqə piyada, akad. ...","[Tibb, Universitetindən, 10, dəqiqə, piyada, ,..."
5,Yeni tikili,6 / 16,77,2,var,800,AZN,ay,"['Nəriman Nərimanov m.', ' Nərimanov r.']",https://bina.az/items/3969594,Nerimanov m yani.heyder eliyev merkezinin yani...,"[Nerimanov, m, yani.heyder, eliyev, merkezinin..."
6,Yeni tikili,2 / 5,52,2,var,900,AZN,ay,"['Səbail r.', ' Şıxov q.']",https://bina.az/items/4002521,"Səbail rayonu, Şıxovda yeni tikilmiş Sea Life ...","[Səbail, rayonu, ,, Şıxovda, yeni, tikilmiş, S..."
7,Yeni tikili,7 / 14,240,5,var,4000,AZN,ay,"['Fontanlar bağı', ' Mərkəzi Univermaq', ' A...",https://bina.az/items/3706981,NİZAMİ REZİDENCE elit binada 5 otaqlı 240m2 t...,"[NİZAMİ, REZİDENCE, elit, binada, , 5, otaqlı..."
8,Yeni tikili,10 / 16,60,2,var,750,AZN,ay,"['Şah İsmayıl Xətai m.', ' Xətai r.']",https://bina.az/items/3949455,Xətai metrosunun cıxışında yerləşir. 2otaql;sə...,"[Xətai, metrosunun, cıxışında, yerləşir, ., 2o..."
9,Yeni tikili,13 / 17,96,3,var,600,AZN,ay,"['Həzi Aslanov m.', ' Xətai r.', ' Həzi Aslano...",https://bina.az/items/3973401,"Həzi Aslanov metronun çıxışında, tam əşşalı ,...","[Həzi, Aslanov, metronun, çıxışında, ,, , tam..."


In [7]:
# kirayeler.write_dataframe(kiraye_evler.head(1))

1 rows successfully written (79IZTmdbrp)


In [11]:
# kirayeler = dataiku.Dataset("KIRAYELER")
# orig_df = kirayeler.get_dataframe()
# final_df = pd.concat([orig_df,kiraye_evler])
# kirayeler.write_with_schema(final_df)


11766 rows successfully written (3n7AzhvI7S)


In [3]:
url='https://bina.az/items/4005651'

headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                              '(KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
            }

requests.get(url, headers=headers)

<Response [200]>